# 01 — Data Cleaning & Feature Engineering

### 🎯 Business Objective
Digital transactions carry fraud risk that increases business losses. To detect and prevent fraudulent events efficiently, we will clean and engineer features to build a reliable fraud detection model.

**Output of this notebook:**  
A processed, feature-engineered dataset saved to  
`data/processed/onlinefraud_clean.csv`


In [ ]:
import os
import sys

# Make project root importable (so `src` can be found)
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from src.features import (
    add_party_type_features,
    add_balance_features,
    encode_categoricals,
    drop_leakage_columns,
)

RAW_PATH = "../data/raw/onlinefraud.csv"
PROCESSED_PATH = "../data/processed/onlinefraud_clean.csv"

pd.set_option("display.max_columns", None)


In [ ]:
df = pd.read_csv(RAW_PATH)
print("Raw data shape:", df.shape)
df.head()




Raw data shape: (6362620, 11)


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [15]:
df.info()
df.describe().T


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


,count,mean,std,min,25%,50%,75%,max
step,6362620.0,2.433972e+02,1.423320e+02,1.0,156.00,239.000,3.350000e+02,7.430000e+02
amount,6362620.0,1.798619e+05,6.038582e+05,0.0,13389.57,74871.940,2.087215e+05,9.244552e+07
oldbalanceOrg,6362620.0,8.338831e+05,2.888243e+06,0.0,0.00,14208.000,1.073152e+05,5.958504e+07
newbalanceOrig,6362620.0,8.551137e+05,2.924049e+06,0.0,0.00,0.000,1.442584e+05,4.958504e+07
oldbalanceDest,6362620.0,1.100702e+06,3.399180e+06,0.0,0.00,132705.665,9.430367e+05,3.560159e+08
newbalanceDest,6362620.0,1.224996e+06,3.674129e+06,0.0,0.00,214661.440,1.111909e+06,3.561793e+08
isFraud,6362620.0,1.290820e-03,3.590480e-02,0.0,0.00,0.000,0.000000e+00,1.000000e+00
isFlaggedFraud,6362620.0,2.514687e-06,1.585775e-03,0.0,0.00,0.000,0.000000e+00,1.000000e+00


In [18]:
print("Missing values per column:")
print(df.isna().sum())

print("\nFraud distribution (%):")
print(df["isFraud"].value_counts(normalize=True) * 100)


Missing values per column:
step               0
type               0
amount             0
nameOrig           0
oldbalanceOrg      0
newbalanceOrig     0
nameDest           0
oldbalanceDest     0
newbalanceDest     0
isFraud            0
isFlaggedFraud     0
orig_party_type    0
dest_party_type    0
dtype: int64

Fraud distribution (%):
isFraud
0    99.870918
1     0.129082
Name: proportion, dtype: float64


In [19]:
# 1. Add customer / merchant roles for origin and destination
df = add_party_type_features(df)

# 2. Add balance movement features
df = add_balance_features(df)

# Quick peek at new columns
df[[
    "nameOrig", "orig_party_type",
    "nameDest", "dest_party_type",
    "oldbalanceOrg", "newbalanceOrig", "orig_balance_change",
    "oldbalanceDest", "newbalanceDest", "dest_balance_change", "net_balance_change"
]].head()


,nameOrig,orig_party_type,nameDest,dest_party_type,oldbalanceOrg,newbalanceOrig,orig_balance_change,oldbalanceDest,newbalanceDest,dest_balance_change,net_balance_change
0,C1231006815,customer,M1979787155,merchant,170136.0,160296.36,-9839.64,0.0,0.0,0.0,-9839.64
1,C1666544295,customer,M2044282225,merchant,21249.0,19384.72,-1864.28,0.0,0.0,0.0,-1864.28
2,C1305486145,customer,C553264065,customer,181.0,0.00,-181.00,0.0,0.0,0.0,-181.00
3,C840083671,customer,C38997010,customer,181.0,0.00,-181.00,21182.0,0.0,-21182.0,-21363.00
4,C2048537720,customer,M1230701703,merchant,41554.0,29885.86,-11668.14,0.0,0.0,0.0,-11668.14


In [20]:
df_encoded = encode_categoricals(df)

print("After encoding categoricals:")
print(df_encoded.shape)
df_encoded.head()


After encoding categoricals:
(6362620, 18)


,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,orig_balance_change,dest_balance_change,net_balance_change,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,dest_party_type_merchant
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,-9839.64,0.0,-9839.64,False,False,True,False,True
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,-1864.28,0.0,-1864.28,False,False,True,False,True
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,-181.00,0.0,-181.00,False,False,False,True,False
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,-181.00,-21182.0,-21363.00,True,False,False,False,False
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,-11668.14,0.0,-11668.14,False,False,True,False,True


In [21]:
df_clean = drop_leakage_columns(df_encoded)

print("After dropping ID columns:")
print(df_clean.shape)
df_clean.head()


After dropping ID columns:
(6362620, 16)


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,orig_balance_change,dest_balance_change,net_balance_change,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,dest_party_type_merchant
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,-9839.64,0.0,-9839.64,False,False,True,False,True
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,-1864.28,0.0,-1864.28,False,False,True,False,True
2,1,181.00,181.0,0.00,0.0,0.0,1,0,-181.00,0.0,-181.00,False,False,False,True,False
3,1,181.00,181.0,0.00,21182.0,0.0,1,0,-181.00,-21182.0,-21363.00,True,False,False,False,False
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,-11668.14,0.0,-11668.14,False,False,True,False,True


In [22]:
df_clean.info()

print("\nFraud distribution in cleaned data (%):")
print(df_clean["isFraud"].value_counts(normalize=True) * 100)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 16 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   step                      int64  
 1   amount                    float64
 2   oldbalanceOrg             float64
 3   newbalanceOrig            float64
 4   oldbalanceDest            float64
 5   newbalanceDest            float64
 6   isFraud                   int64  
 7   isFlaggedFraud            int64  
 8   orig_balance_change       float64
 9   dest_balance_change       float64
 10  net_balance_change        float64
 11  type_CASH_OUT             bool   
 12  type_DEBIT                bool   
 13  type_PAYMENT              bool   
 14  type_TRANSFER             bool   
 15  dest_party_type_merchant  bool   
dtypes: bool(5), float64(8), int64(3)
memory usage: 564.3 MB

Fraud distribution in cleaned data (%):
isFraud
0    99.870918
1     0.129082
Name: proportion, dtype: float64


In [23]:
os.makedirs(os.path.dirname(PROCESSED_PATH), exist_ok=True)

df_clean.to_csv(PROCESSED_PATH, index=False)
print("Saved cleaned dataset to:", PROCESSED_PATH)


Saved cleaned dataset to: ../data/processed/onlinefraud_clean.csv
